In [104]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler


# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")

data = data.drop("date", axis=1)

data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()

data = data.astype('float32')


# Keep data until 31.08.2023
data = data.iloc[:10747]

#print(data['open'].dtype)
#print(data.shape)

# Assuming 'data' is a pandas DataFrame
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']]
y_data = data["close"]

# Now x_data and y_data are pandas DataFrames/Series, respectively

x_data.tail(1)

,open,high,low,volume,adjusted_close,change_percent,avg_vol_20d
10746,196.059998,196.490005,195.259995,38824100.0,196.185104,0.32,49803320.0


In [105]:
# Set the window size for training
train_window_size = 20

# Initialize lists to store training and temporary sets
x_train_list, y_train_list, x_temp_list, y_temp_list = [], [], [], []

# Iterate through the data with the specified window size
for i in range(0, len(x_data) - train_window_size, train_window_size + 1):
    x_train_temp = x_data.iloc[i:i+train_window_size+1]
    y_train_temp = y_data.iloc[i:i+train_window_size+1]

    # Separate the last row for the temporary set
    x_train = x_train_temp.iloc[:-1]
    y_train = y_train_temp.iloc[:-1]

    x_temp = x_train_temp.iloc[-1:]
    y_temp = y_train_temp.iloc[-1:]

    x_train_list.append(x_train)
    y_train_list.append(y_train)
    x_temp_list.append(x_temp)
    y_temp_list.append(y_temp)

# Concatenate the lists into pandas DataFrames
x_train = pd.concat(x_train_list)
y_train = pd.concat(y_train_list)
x_temp = pd.concat(x_temp_list)
y_temp = pd.concat(y_temp_list)

# print(y_train.head(50))
x_temp_train, x_temp_val, y_temp_train, y_temp_val = train_test_split(x_temp, y_temp, test_size=0.2, random_state=42)


# Split x_temp and y_temp into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=0.5, random_state=42)


In [106]:
from sklearn.preprocessing import StandardScaler

# Assuming x_train, x_temp_train, and x_temp_val are DataFrames containing your features

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler on x_train and transform x_train, x_temp_train, and x_temp_val
x_train_scaled = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)

x_test_scaled = pd.DataFrame(scaler.transform(x_test), columns=x_temp_val.columns)

x_val_scaled = pd.DataFrame(scaler.transform(x_val), columns=x_temp_train.columns)

In [107]:
learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 4
batch_evaluation_frequency = 10
epochs = len(x_train) - window_size
batch_size = 1
input_size = 7  # Number of features
output_size = 7  # Number of features to predict

class LSTMModel7(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel7, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden




In [108]:
# Instantiate the model
model7 = LSTMModel7(input_size, hidden_dim, n_layers, output_size)

# Choose a loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model7.parameters(), lr=learning_rate)


epochs = 1
window_size = 20

# Training loop (use model7 instead of model)
for epoch in range(epochs):
    for i in range(0, len(x_train) - window_size, batch_size):
        inputs = x_train[i:i+window_size]
        targets = y_train[i+window_size]

        # Convert inputs and targets to PyTorch tensors
        inputs = torch.tensor(inputs, dtype=torch.float32)
        targets = torch.tensor(targets, dtype=torch.float32)

        # Zero the gradients
        optimizer.zero_grad()

        # Initialize hidden state
        hidden = model7.init_hidden(batch_size)

        # Forward pass
        outputs, _ = model7(inputs.unsqueeze(0), hidden)
        loss = criterion(outputs.squeeze(), targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print loss or other metrics occasionally
        if i % batch_evaluation_frequency == 0:
            print(f'Epoch {epoch}, Batch {i}, Loss: {loss.item()}')

KeyError: 20